In [32]:
%store -r model_data_a
%store -r model_data_b
%store -r target_repository_uri_a
%store -r target_repository_uri_b

In [33]:
image_uri_a = f"{target_repository_uri_a}:1"
image_uri_b = f"{target_repository_uri_b}:1"

In [34]:
container1 = { 
    'Image': image_uri_a,
    'ContainerHostname': 'containerA',
    'ModelDataUrl': model_data_a
}

container2 = { 
    'Image': image_uri_b,
    'ContainerHostname': 'containerB',
    'ModelDataUrl': model_data_b
}

execution_config = {'Mode': 'Direct'}

In [35]:
model_name = "multi-container"
endpoint_config_name = 'multi-container-config'
endpoint_name = 'multi-container-endpoint'

In [36]:
import boto3
import sagemaker

sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()
sm_client = boto3.Session().client('sagemaker')

In [37]:
# sm_client.delete_endpoint(EndpointName=endpoint_name)
# sm_client.delete_endpoint_config(EndpointConfigName=endpoint_config_name)
# sm_client.delete_model(ModelName=model_name)

In [38]:
response = sm_client.create_model(
    ModelName = model_name,
    InferenceExecutionConfig = execution_config,
    ExecutionRoleArn = role,
    Containers = [container1, container2])

response

{'ModelArn': 'arn:aws:sagemaker:us-east-1:581320662326:model/multi-container',
 'ResponseMetadata': {'RequestId': '4b4a451c-97ca-4bf2-b745-84921ba925b2',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '4b4a451c-97ca-4bf2-b745-84921ba925b2',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '77',
   'date': 'Sat, 01 May 2021 13:57:20 GMT'},
  'RetryAttempts': 0}}

In [39]:
response = sm_client.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[{
        'InstanceType': 'ml.t2.medium',
        'InitialInstanceCount': 1,
        'InitialVariantWeight': 1,
        'ModelName': model_name,
        'VariantName': 'AllTraffic'}])

response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:581320662326:endpoint-config/multi-container-config',
 'ResponseMetadata': {'RequestId': 'dc0e4072-f3b1-4276-adcb-45eab9729477',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'dc0e4072-f3b1-4276-adcb-45eab9729477',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '103',
   'date': 'Sat, 01 May 2021 13:57:20 GMT'},
  'RetryAttempts': 0}}

In [40]:
response = sm_client.create_endpoint(
    EndpointName       = endpoint_name,
    EndpointConfigName = endpoint_config_name)

response

{'EndpointArn': 'arn:aws:sagemaker:us-east-1:581320662326:endpoint/multi-container-endpoint',
 'ResponseMetadata': {'RequestId': '1a3d6e84-0eb1-4036-8ef0-01e8dc26b65d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '1a3d6e84-0eb1-4036-8ef0-01e8dc26b65d',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '92',
   'date': 'Sat, 01 May 2021 13:57:20 GMT'},
  'RetryAttempts': 0}}

In [41]:
response = sm_client.describe_endpoint(
    EndpointName=endpoint_name
)

response['EndpointStatus']

'Creating'

In [42]:
from time import sleep

while response['EndpointStatus'] == "Creating":
    print("Still creating...")
    sleep(60)
    
    response = sm_client.describe_endpoint(
        EndpointName=endpoint_name
    )

Still creating...
Still creating...
Still creating...
Still creating...
Still creating...
Still creating...
Still creating...
Still creating...
Still creating...
Still creating...


In [43]:
response

{'EndpointName': 'multi-container-endpoint',
 'EndpointArn': 'arn:aws:sagemaker:us-east-1:581320662326:endpoint/multi-container-endpoint',
 'EndpointConfigName': 'multi-container-config',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'DeployedImages': [{'SpecifiedImage': '581320662326.dkr.ecr.us-east-1.amazonaws.com/cookbook-custom-image-a:1',
     'ResolvedImage': '581320662326.dkr.ecr.us-east-1.amazonaws.com/cookbook-custom-image-a@sha256:0aac309142f6c3b72b8c2d7c36c602279d6074a0ce6d699b6b14f2d079054c37',
     'ResolutionTime': datetime.datetime(2021, 5, 1, 13, 57, 26, 216000, tzinfo=tzlocal())},
    {'SpecifiedImage': '581320662326.dkr.ecr.us-east-1.amazonaws.com/cookbook-custom-image-b:1',
     'ResolvedImage': '581320662326.dkr.ecr.us-east-1.amazonaws.com/cookbook-custom-image-b@sha256:77d096aafa00bdc613d4b542eb8a2b33a30b3934487c19e121d31ddb591f6d81',
     'ResolutionTime': datetime.datetime(2021, 5, 1, 13, 57, 26, 259000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
  

In [44]:
runtime_sm_client = boto3.client('sagemaker-runtime')

In [45]:
body = "42"

response = runtime_sm_client.invoke_endpoint(
                        EndpointName = endpoint_name,
                        ContentType  = 'text/csv',
                        TargetContainerHostname='containerA',
                        Body         = body)

response['Body'].read()

b'1233.18227456514'

In [46]:
response = runtime_sm_client.invoke_endpoint(
                        EndpointName = endpoint_name,
                        ContentType  = 'text/csv',
                        TargetContainerHostname='containerB',
                        Body         = body)

response['Body'].read()

b'1233.1822745651407'

In [47]:
response = sm_client.delete_endpoint(
    EndpointName=endpoint_name
)

response

{'ResponseMetadata': {'RequestId': '33d038ee-a336-4405-af98-d978702ab3f5',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '33d038ee-a336-4405-af98-d978702ab3f5',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0',
   'date': 'Sat, 01 May 2021 14:07:23 GMT'},
  'RetryAttempts': 0}}